In [ ]:
from pathlib import Path

import polars as pl
import yaml
from pam import read

from nhts.trips import preprocess

In [ ]:
pops = {}

with open("nhts/trip_dictionary.yaml") as f:
    config = yaml.safe_load(f)

for year, name in [
    (2022, "tripv2pub.csv"),
    (2017, "trippub.csv"),
    (2009, "DAYV2PUB.CSV"),
    (2001, "DAYPUB.csv")
    ]:
    path = Path("~/Data/foundata/NHTS") / str(year) / name
    trips = (
        pl.read_csv(path, ignore_errors=True)
    )
    trips = preprocess(trips, year=year, config=config)
    print(len(trips))
    # pam_trips = read.load_travel_diary(
    #     trips=trips.to_pandas()
    # )
    # pops[year] = pam_trips


30844
915050
1060185
472704


In [ ]:
root = Path("~/Data/foundata/NHTS/2022")
codesheet = pl.read_excel(root / "codebook.xlsx", sheet_name="Household")

codesheet = codesheet.with_columns([
    pl.when(pl.col(c) == "").then(None).otherwise(pl.col(c)).alias(c)
    for c in codesheet.columns if codesheet[c].dtype == pl.String
]).fill_null(strategy="forward").filter(pl.col("Type") == "C").with_columns([
    pl.col("Code / Range").str.splitn("=", 2).struct.field("field_0").alias("key"),
    pl.col("Code / Range").str.splitn("=", 2).struct.field("field_1").alias("value")
]).filter(pl.col("key").cast(pl.Int64, strict=False).is_not_null()).with_columns(
    key = pl.col("key").cast(pl.Int32)
)
for i, frame in codesheet.group_by("Name"):
    

Name,Label,Type,Length,Code / Range,Frequency,Weighted,key,value
str,str,str,i64,str,i64,f64,i32,str
"""CDIVMSAR""","""Grouping of household by combi…","""C""",2,"""11=New England MSA/CMSA of 1 m…",161,2.0662e6,11,"""New England MSA/CMSA of 1 mill…"
"""CDIVMSAR""","""Grouping of household by combi…","""C""",2,"""12=New England MSA/CMSA of 1 m…",51,703523.166832,12,"""New England MSA/CMSA of 1 mill…"
"""CDIVMSAR""","""Grouping of household by combi…","""C""",2,"""13=New England MSA of less tha…",208,3.1351e6,13,"""New England MSA of less than 1…"
"""CDIVMSAR""","""Grouping of household by combi…","""C""",2,"""14=New England Not in an MSA""",19,221998.326625,14,"""New England Not in an MSA"""
"""CDIVMSAR""","""Grouping of household by combi…","""C""",2,"""21=Mid-Atlantic MSA/CMSA of 1 …",468,8.3308e6,21,"""Mid-Atlantic MSA/CMSA of 1 mil…"
…,…,…,…,…,…,…,…,…
"""URBANSIZE""","""Urban area size where home add…","""C""",2,"""06=Not in urbanized area""",2261,3.5613e7,6,"""Not in urbanized area"""
"""URBRUR""","""Household in urban/rural area""","""C""",2,"""01=Urban""",6306,1.0292e8,1,"""Urban"""
"""URBRUR""","""Household in urban/rural area""","""C""",2,"""02=Rural""",1587,2.4625e7,2,"""Rural"""


In [ ]:
root = Path("~/Data/foundata/NHTS/2022") / str(year) / name

for sheet in zip(
    ["Household", "Person", "Vehicle", "Trip", "Long Distance"],
    ["hh", "person", "veh", "trip", "long"]
):
    map = pl.read_excel(root / "codebook.xlsx", sheet_name=sheet)

# for table, name in zip(
#     ["HOUSEHOLD", "PERSON", "GPS_DAY", "VEHICLE"],
#     
# ):
#     data = maps.filter(pl.col("TABLE") == table)
#     mapper = {}
#     for i, frame in data.group_by("NAME"):
#         column = str(i[0]).lower()
#         mapper[column] = {}
#         keys = frame.select(pl.col("VALUE")).rows()
#         values = frame.select(pl.col("LABEL")).rows()
#         for k, v in zip(keys, values):
#             mapper[column][int(k[0])] = str(v[0])

#     with open(f"cmap/{name}_dictionary.json", "w", encoding="utf-8") as f:
#         json.dump(mapper, f, ensure_ascii=False, indent=4)

In [ ]:
map

Name,Label,Type,Length,Code / Range,Frequency,Weighted
str,str,str,i64,str,i64,f64
"""CDIVMSAR""","""Grouping of household by combi…","""C""",2,"""11=New England MSA/CMSA of 1 m…",161,2.0662e6
"""""",null,"""""",null,"""12=New England MSA/CMSA of 1 m…",51,703523.166832
"""""",null,"""""",null,"""13=New England MSA of less tha…",208,3.1351e6
"""""",null,"""""",null,"""14=New England Not in an MSA""",19,221998.326625
"""""",null,"""""",null,"""21=Mid-Atlantic MSA/CMSA of 1 …",468,8.3308e6
…,…,…,…,…,…,…
"""YOUNGCHILD""","""Count of household members und…","""N""",8,"""0""",7271,1.1434e8
"""""",null,"""""",null,"""1""",445,9.4967e6
"""""",null,"""""",null,"""2""",154,3.2843e6


In [ ]:
hhs

HOUSEID,WTHHFIN,WTHHFIN5D,WTHHFIN2D,NUMADLT,HOMEOWN,HOMETYPE,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CNTTDHH,CDIVMSAR,FLAG100,HHFAMINC,HHFAMINC_IMP,HH_RACE,HHSIZE,HHVEHCNT,HHRELATD,LIF_CYC,MSACAT,MSASIZE,TRAVDAY,URBAN,URBANSIZE,URBRUR,PPT517,YOUNGCHILD,RESP_CNT,URBRUR_2010,TDAYDATE,WRKCOUNT,STRATUMID,HBHUR,HTHTNRNT,HTPPOPDN,HTRESDN,HTEEMPDN,HBHTNRNT,HBPPOPDN,HBRESDN
i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
9000013002,4621.758002,0.0,13946.191168,2,1,1,1,1,1,2,2,7,11,1,11,11,1,4,2,1,4,1,5,1,1,4,1,2,0,4,1,202202,1,1021,4,20,4,2,5,5,4,3
9000013016,2982.998407,3878.814279,0.0,2,3,3,2,1,1,2,2,6,13,1,7,7,1,2,1,1,2,3,3,4,1,2,1,0,0,2,1,202202,2,1021,3,80,7,3,8,90,7,6
9000013026,9979.573402,14700.796695,0.0,1,3,3,1,1,1,2,1,0,11,1,10,10,1,1,0,2,1,1,5,3,1,4,1,0,0,1,1,202202,1,1021,5,60,7,6,8,70,7,3
9000013039,10355.890735,13150.678996,0.0,2,3,3,1,1,1,2,2,2,11,1,7,7,1,4,1,1,6,1,4,5,1,4,1,2,0,4,1,202201,1,1021,5,60,7,6,8,60,7,2
9000013041,4926.862529,7061.357485,0.0,1,1,2,2,1,1,2,1,0,12,1,11,11,1,1,1,2,1,2,4,2,1,3,1,0,0,1,1,202201,1,1021,4,30,5,4,6,40,6,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9000218013,8792.888034,11731.186218,0.0,2,1,1,2,6,3,2,1,3,63,1,3,3,1,2,1,1,10,3,1,5,1,1,1,0,0,2,1,202301,0,1031,2,20,4,3,4,5,3,2
9000218021,2508.248447,0.0,16373.594134,1,2,1,2,6,3,2,1,0,63,1,4,4,1,1,1,2,9,3,1,1,4,6,2,0,0,1,2,202212,0,1032,2,5,2,2,3,20,2,2
9000218034,12559.6319,17007.945486,0.0,2,2,1,2,6,3,2,2,0,64,1,4,4,1,2,2,1,10,4,6,5,4,6,2,0,0,2,2,202212,0,1032,1,5,1,1,-9,5,1,1
